In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install vadersentiment
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 38.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 9.3 MB/s 
     |████████████████████████████████| 662 kB 66.2 MB/s 
     |████████████████████████████████| 85 kB 5.8 MB/s 
     |████████████████████████████████| 88 kB 8.8 MB/s 
     |████████████████████████████████| 5.2 MB 60.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 5.8 MB 68.1 MB/s 
     |████████████████████████████████| 1.3 MB 73.4 MB/s 
     |████████████████████████████████| 182 kB 83.5 MB/s 
     |████████████████████████████████| 7.6 MB 69.6 MB/s 
     |████████████████████████████████| 1.1 MB 72.8 MB/s 
  Created wheel for hdbscan: filename=hdbsca

In [3]:
import pandas as pd
import numpy as np
import gensim
from nltk.stem import WordNetLemmatizer, SnowballStemmer
np.random.seed(10)
import nltk
from bertopic import BERTopic
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import seaborn as sns
import plotly.express as px
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [4]:
import pandas as pd
file_name = "/content/drive/MyDrive/raw_reddit_politics_posts_comments.csv"
df = pd.read_csv(file_name)
df.head()

,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,comments
0,j767z3,politics,I_Mispelled_Mispell,2020-10-08 03:59:21,buzzfeednews.com,https://www.buzzfeednews.com/article/stephanie...,Kamala Harris's Energy Towards Pence's Interru...,29,NaN,1,I would bet the writer is white....What the fu...
1,j76abt,politics,Xeelee1123,2020-10-08 04:03:49,theguardian.com,https://www.theguardian.com/us-news/2020/oct/0...,Trump calls Covid diagnosis 'blessing from God...,29,NaN,1,Hi `Xeelee1123`. Thank you for participating i...
2,j76cwv,politics,slagholer,2020-10-08 04:08:46,talkingpointsmemo.com,https://talkingpointsmemo.com/news/pence-remin...,Pence Reminds Us All That The Pre-Trump GOP Is...,71,NaN,1,"But you guys, Trump has Jewish grandchildren. ..."
3,j76e7j,politics,NotsoPG,2020-10-08 04:11:19,democracynow.org,https://www.democracynow.org/2020/10/7/headlin...,"NYT: Jeff Sessions Defended Child Separations,...",14,NaN,1,He got better at picking the worst people over...
4,j76eku,politics,0belvedere,2020-10-08 04:12:05,nytimes.com,https://www.nytimes.com/2020/10/07/us/politics...,A fly sat atop Mike Pence's head for two minut...,44,NaN,1,well put. thanks....The fly explains its think...


In [5]:
rel_cols = list(df.columns)

In [6]:
file_name1 = "/content/drive/MyDrive/raw_reddit_politics_posts_comments_2.csv"
df2 = pd.read_csv(file_name1)
df2.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7,9,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,...,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143
0,jkek6x,politics,trifecta,29-10-2020 17:33,wfla.com,https://www.wfla.com/news/florida/florida-woma...,Florida woman in labor makes husband stop at e...,314,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,jkek7q,politics,freebandgang25,29-10-2020 17:33,cnn.com,https://www.cnn.com/2020/10/29/politics/florid...,Biden and Trump head to Florida for dueling ra...,7,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,jkel2l,politics,Mason0001,29-10-2020 17:35,bbc.com,https://www.bbc.com/news/amp/world-54553132,Hunter Biden: What was he doing in Ukraine and...,47,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,jkelpw,politics,freebandgang25,29-10-2020 17:35,cnn.com,https://www.cnn.com/2020/10/29/politics/f-35-j...,Top Democrat says Trump administration moving ...,18,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,jkelt9,politics,MadeInOne1,29-10-2020 17:36,commondreams.org,https://www.commondreams.org/views/2020/10/29/...,Let's Be Very Clear: Senate Republicans Have F...,308,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df3 = df2[rel_cols]

In [16]:
df3.shape

(74256, 11)

In [8]:
df_final = pd.concat([df,df3],ignore_index = True)

In [16]:
df_final.shape

(87656, 11)

In [17]:
#df_final.to_csv('/content/drive/MyDrive/raw_reddit_politics_posts_and_comments',index = False)

In [17]:
df = df_final.copy()

In [10]:
def clean_text(text):
    text = text.lower() # lowercase everything
    text = text.encode('ascii', 'ignore').decode()  # remove unicode characters
    text = re.sub(r'https*\S+', ' ', text) # remove links
    text = re.sub(r'http*\S+', ' ', text)
    # cleaning up text
    text = re.sub(r'\'\w+', '', text) 
    text = re.sub(r'\w*\d+\w*', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'\s[^\w\s]\s', '', text)
    text = re.sub("[\(\[].*?[\)\]]", "", text)
    text = re.sub('[^a-zA-Z0-9 \.]', '', text)
    return text

In [11]:
def lemmatize(text):
  return WordNetLemmatizer().lemmatize(text, pos='v')
def preprocess(text):
    if type(text)!=str:
      return ''
    result = []
    text = text.replace('[removed]', '').replace('[deleted]', '')
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(lemmatize(token))
    return ' '.join(result)

In [18]:
print(df.shape)
df.fillna("", inplace=True)
len(df[df.id==""])

(87656, 11)


338

In [19]:
df["title"] = df["title"].astype(str) 
df["selftext"] = df["selftext"].astype(str)
df["title"] = df["title"] + " " + df["selftext"]

In [24]:
df['created_utc'] = pd.to_datetime(df['created_utc'],errors = "coerce")
df["created_dt_month"] = df['created_utc'].dt.strftime('%m-%Y')


In [25]:
df['cleaned_title'] = df.title.apply(preprocess)
df = df.dropna()
titles = df["cleaned_title"].astype(str).to_list()

In [26]:
len(titles)

87100

In [27]:
topic_model = BERTopic(min_topic_size=30, n_gram_range=(1,3), nr_topics='auto', verbose=True)
topics, _ = topic_model.fit_transform(titles)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/2722 [00:00<?, ?it/s]

2022-12-02 18:25:02,064 - BERTopic - Transformed documents to Embeddings
2022-12-02 18:27:17,945 - BERTopic - Reduced dimensionality
2022-12-02 18:27:27,474 - BERTopic - Clustered reduced embeddings
2022-12-02 18:27:45,501 - BERTopic - Reduced number of topics from 517 to 392


In [29]:
topic_model.save("/content/drive/MyDrive/topic_politics_model")

/usr/local/lib/python3.8/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [4]:
# topic_model = BERTopic.load("/content/drive/MyDrive/topic_politics_model")

In [28]:
barchart = topic_model.visualize_barchart(top_n_topics=10, n_words=10, height=400)
barchart

In [30]:
barchart.write_html("/content/drive/MyDrive/politics_word_freq.html")

In [31]:
intertopic_dist = topic_model.visualize_topics()
intertopic_dist

In [32]:
intertopic_dist.write_html("/content/drive/MyDrive/politics_intertopic_distance.html")

In [33]:
freq = topic_model.get_topic_info()
freq.head(10)

,Topic,Count,Name
0,-1,31409,-1_trump_biden_capitol_election
1,0,2600,0_covid_coronavirus_relief_covid relief
2,1,1582,1_georgia_runoffs_georgia senate_senate runoffs
3,2,1388,2_impeachment_graham_lindsey graham_lindsey
4,3,1208,3_vaccine_covid vaccine_pfizer_vaccines
5,4,1109,4_iran_china_nuclear_israel
6,5,1045,5_wisconsin_court_supreme court_supreme
7,6,1016,6_michigan_certify_board_vote certify
8,7,798,7_joe biden_joe_biden_biden win
9,8,766,8_fraud_voter fraud_voter_election fraud


In [34]:
topic_nr = freq.iloc[3]["Topic"]  # We select a frequent topic
topic_model.get_topic(topic_nr)

[('impeachment', 0.0305512725864422),
 ('graham', 0.02691797864310998),
 ('lindsey graham', 0.021748126170747412),
 ('lindsey', 0.02172787287476091),
 ('impeach', 0.018596358897804935),
 ('trial', 0.01743875483726406),
 ('impeachment trial', 0.015111515480410337),
 ('trump impeachment', 0.013988861543296548),
 ('impeach trump', 0.010205729560910107),
 ('mcconnell', 0.00906807387314789)]

In [35]:
freq["word_strength"] = freq.Topic.apply(topic_model.get_topic)
freq.head()

,Topic,Count,Name,word_strength
0,-1,31409,-1_trump_biden_capitol_election,"[(trump, 0.002485019340804115), (biden, 0.0024..."
1,0,2600,0_covid_coronavirus_relief_covid relief,"[(covid, 0.01742532521435388), (coronavirus, 0..."
2,1,1582,1_georgia_runoffs_georgia senate_senate runoffs,"[(georgia, 0.036043579556746946), (runoffs, 0...."
3,2,1388,2_impeachment_graham_lindsey graham_lindsey,"[(impeachment, 0.0305512725864422), (graham, 0..."
4,3,1208,3_vaccine_covid vaccine_pfizer_vaccines,"[(vaccine, 0.05112016886766617), (covid vaccin..."


In [36]:
freq.to_csv("/content/drive/MyDrive/politics_topic_frequency.csv", index=False)

In [37]:
# df = df.dropna()
# df['created_utc'] = pd.to_datetime(df['created_utc'])
# df["created_dt_month"] = df['created_utc'].dt.strftime('%m-%Y')

titles = df["cleaned_title"].astype(str)
dates = df['created_utc']

topics_over_time = topic_model.topics_over_time(docs=titles, timestamps=dates, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=30)

26it [05:26, 12.55s/it]


In [67]:
len(titles)

87100

In [38]:
tot = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)
tot

In [39]:
tot.write_html("/content/drive/MyDrive/politics_topics_over_time.html")

In [40]:
topics_over_time.to_csv("/content/drive/MyDrive/topics_over_time_politics.csv", index=False)
representative_docs = topic_model.get_representative_docs()

docs_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in representative_docs.items() ])).T
docs_df.index.name = "topic_nr"
docs_df.to_csv("/content/drive/MyDrive/representative_docs_politics.csv")
docs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
topic_nr,,,,,,,,,,,,,,,,,,,,,
21,pence call harris trump continue stew denial,fact check vice presidential debate mike pence...,rick santorum interrupt female colleague argue...,man home future hold vice president mike pence,mike pence homeless,trump stupidest possible argument mike pence,pence adviser obst catch virus add white house...,pence politics regularly seep coronavirus task...,pence travel north carolina despite contact in...,politifact constitution allow mike pence rejec...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,judge limit california governor newsom power c...,top daily deaths coronavirus time,record day covid deaths trump falsely proclaim...,biden covid board include whistleblower say tr...,trump face criticism rag covid crisis delay bi...,biden want actual scientists advise covid,nancy pelosi defend reject trump administratio...,pelosi defiant handle stimulus reject previous...,trump approve stimulus mnuchin pelosi talk lat...,stimulus check likely approve mid march experts,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,trump good response question kid separate pare...,lawyers parent migrant children separate trump...,biden sign executive order immigration include...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
377,fly sit mike pence head minutes steal debate,biden campaign start sell fly swatter right de...,history remember mike pence performance reason...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,trump team propose rule stall biden health car...,donald trump health care measure joe biden lik...,biden lift health care plan insurance industry...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df['topic'] = df['cleaned_title'].apply(lambda x: topic_model.find_topics(x)[0][0])
df.head()

,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,comments,cleaned_title,created_dt_month,topic
0,j767z3,politics,I_Mispelled_Mispell,2020-10-08 03:59:21,buzzfeednews.com,https://www.buzzfeednews.com/article/stephanie...,Kamala Harris's Energy Towards Pence's Interru...,29,,1,I would bet the writer is white....What the fu...,kamala harris energy pence interruptions debat...,10-2020,37
1,j76abt,politics,Xeelee1123,2020-10-08 04:03:49,theguardian.com,https://www.theguardian.com/us-news/2020/oct/0...,Trump calls Covid diagnosis 'blessing from God...,29,,1,Hi `Xeelee1123`. Thank you for participating i...,trump call covid diagnosis bless god amid fals...,10-2020,26
2,j76cwv,politics,slagholer,2020-10-08 04:08:46,talkingpointsmemo.com,https://talkingpointsmemo.com/news/pence-remin...,Pence Reminds Us All That The Pre-Trump GOP Is...,71,,1,"But you guys, Trump has Jewish grandchildren. ...",pence remind pre trump gop alive kick terminal...,10-2020,21
3,j76e7j,politics,NotsoPG,2020-10-08 04:11:19,democracynow.org,https://www.democracynow.org/2020/10/7/headlin...,"NYT: Jeff Sessions Defended Child Separations,...",14,,1,He got better at picking the worst people over...,nyt jeff sessions defend child separations say...,10-2020,71
4,j76eku,politics,0belvedere,2020-10-08 04:12:05,nytimes.com,https://www.nytimes.com/2020/10/07/us/politics...,A fly sat atop Mike Pence's head for two minut...,44,,1,well put. thanks....The fly explains its think...,fly sit atop mike pence head minutes debate,10-2020,377


In [42]:
df['cleaned_comments'] = df['comments'].map(preprocess)
df.to_csv("/content/drive/MyDrive/reddit_politics_data_with_topics.csv", index=False)
df.head()

,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,comments,cleaned_title,created_dt_month,topic,cleaned_comments
0,j767z3,politics,I_Mispelled_Mispell,2020-10-08 03:59:21,buzzfeednews.com,https://www.buzzfeednews.com/article/stephanie...,Kamala Harris's Energy Towards Pence's Interru...,29,,1,I would bet the writer is white....What the fu...,kamala harris energy pence interruptions debat...,10-2020,37,bet writer white fuck read read like bad parod...
1,j76abt,politics,Xeelee1123,2020-10-08 04:03:49,theguardian.com,https://www.theguardian.com/us-news/2020/oct/0...,Trump calls Covid diagnosis 'blessing from God...,29,,1,Hi `Xeelee1123`. Thank you for participating i...,trump call covid diagnosis bless god amid fals...,10-2020,26,xeelee thank participate politics submission r...
2,j76cwv,politics,slagholer,2020-10-08 04:08:46,talkingpointsmemo.com,https://talkingpointsmemo.com/news/pence-remin...,Pence Reminds Us All That The Pre-Trump GOP Is...,71,,1,"But you guys, Trump has Jewish grandchildren. ...",pence remind pre trump gop alive kick terminal...,10-2020,21,guy trump jewish grandchildren possibly suppor...
3,j76e7j,politics,NotsoPG,2020-10-08 04:11:19,democracynow.org,https://www.democracynow.org/2020/10/7/headlin...,"NYT: Jeff Sessions Defended Child Separations,...",14,,1,He got better at picking the worst people over...,nyt jeff sessions defend child separations say...,10-2020,71,get better pick worst people time strike gold ...
4,j76eku,politics,0belvedere,2020-10-08 04:12:05,nytimes.com,https://www.nytimes.com/2020/10/07/us/politics...,A fly sat atop Mike Pence's head for two minut...,44,,1,well put. thanks....The fly explains its think...,fly sit atop mike pence head minutes debate,10-2020,377,thank fly explain think https twitter com vinn...


In [43]:
sentiment_analyzer = SentimentIntensityAnalyzer()
df["comment_sentiment"] = df['cleaned_comments'].apply(lambda x: sentiment_analyzer.polarity_scores(x)["compound"])

In [44]:
df.head()

,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,comments,cleaned_title,created_dt_month,topic,cleaned_comments,comment_sentiment
0,j767z3,politics,I_Mispelled_Mispell,2020-10-08 03:59:21,buzzfeednews.com,https://www.buzzfeednews.com/article/stephanie...,Kamala Harris's Energy Towards Pence's Interru...,29,,1,I would bet the writer is white....What the fu...,kamala harris energy pence interruptions debat...,10-2020,37,bet writer white fuck read read like bad parod...,-0.9513
1,j76abt,politics,Xeelee1123,2020-10-08 04:03:49,theguardian.com,https://www.theguardian.com/us-news/2020/oct/0...,Trump calls Covid diagnosis 'blessing from God...,29,,1,Hi `Xeelee1123`. Thank you for participating i...,trump call covid diagnosis bless god amid fals...,10-2020,26,xeelee thank participate politics submission r...,0.9930
2,j76cwv,politics,slagholer,2020-10-08 04:08:46,talkingpointsmemo.com,https://talkingpointsmemo.com/news/pence-remin...,Pence Reminds Us All That The Pre-Trump GOP Is...,71,,1,"But you guys, Trump has Jewish grandchildren. ...",pence remind pre trump gop alive kick terminal...,10-2020,21,guy trump jewish grandchildren possibly suppor...,0.6494
3,j76e7j,politics,NotsoPG,2020-10-08 04:11:19,democracynow.org,https://www.democracynow.org/2020/10/7/headlin...,"NYT: Jeff Sessions Defended Child Separations,...",14,,1,He got better at picking the worst people over...,nyt jeff sessions defend child separations say...,10-2020,71,get better pick worst people time strike gold ...,-0.9914
4,j76eku,politics,0belvedere,2020-10-08 04:12:05,nytimes.com,https://www.nytimes.com/2020/10/07/us/politics...,A fly sat atop Mike Pence's head for two minut...,44,,1,well put. thanks....The fly explains its think...,fly sit atop mike pence head minutes debate,10-2020,377,thank fly explain think https twitter com vinn...,-0.1010


In [45]:
# df['created_utc'] = pd.to_datetime(df['created_utc'],errors = "coerce")
df["created_dt_month"] = df['created_utc'].dt.strftime('%m-%Y')

In [46]:
sentiment_df = df[["created_dt_month", "topic", "comment_sentiment"]].groupby(["created_dt_month", "topic"]).mean()
sentiment_df.head()

comment_sentiment
created_dt_month topic                   
01-2020          0              -0.465488
                 1              -0.313096
                 2              -0.569740
                 3              -0.146867
                 4               0.931500

In [47]:
sentiment_df = sentiment_df.reset_index()

In [ ]:
freq

In [48]:
map_topic = freq[["Topic", "Name"]]
map_topic.columns = ['topic', 'name']
merged_df = sentiment_df.merge(map_topic, on=['topic'])

In [49]:
merged_df["created_dt_month"] = pd.to_datetime(merged_df["created_dt_month"])
merged_df = merged_df.sort_values("created_dt_month")
merged_df = merged_df[merged_df['topic']!=-1]
merged_df.head()

,created_dt_month,topic,comment_sentiment,name
0,2020-01-01,0,-0.465488,0_covid_coronavirus_relief_covid relief
3935,2020-01-01,382,0.930150,382_ice cube_cube_ice_plan black
3941,2020-01-01,383,-0.928000,383_hotel_election night_election night party_...
1619,2020-01-01,110,-0.991300,110_mccarthy_kevin mccarthy_kevin_leader kevin...
1604,2020-01-01,108,-0.982100,108_rittenhouse_kyle rittenhouse_kyle_kenosha


In [50]:
plot_df = merged_df[merged_df['topic'].isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])]
# plot_df["created_dt_month"] = pd.to_datetime(plot_df["created_dt_month"])
fig = px.line(plot_df, x="created_dt_month", y="comment_sentiment", color="name")
fig.show()

In [51]:
fig.write_html("/content/drive/MyDrive/politics_comments_sentiments.html")

In [52]:
merged_df.to_csv("/content/drive/MyDrive/politics_comment_sentiment.csv", index=False)

In [53]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
docs = titles
# topic_model = BERTopic(verbose=True, n_gram_range=(1, 3),nr_topics='auto',min_topic_size=100)
# topics, _ = topic_model.fit_transform(docs)

# Preprocess Documents
documents = pd.DataFrame({"Document": docs,
                          "ID": range(len(docs)),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()
print(coherence)

0.2550903681336496
